<center><a href="https://www.nvidia.com/dli"> <img src="https://drive.google.com/uc?id=1J3bpGyJcz-7uOFkUNhvOBiReBCk-sUwR" width="200" </a></center>

#Conectando ao Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/lab 6
!ls

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Colab Notebooks/lab 6'
/content
gdrive	sample_data


#Classificação de Imagens da base de dados de Linguagem Americana de Sinais

Nesta seção, realizaremos as etapas de preparação de dados, criação de modelo e treinamento de modelo que observamos na última seção usando um conjunto de dados diferente: imagens de mãos fazendo letras em [Linguagem de Sinais Americana](http://www.asl.gs /).

## Objectives

* Prepare dados de imagem para treinamento
* Crie e compile um modelo simples para classificação de imagens
* Treine um modelo de classificação de imagens e observe os resultados

## American Sign Language Dataset

O [alfabeto da linguagem de sinais americana](http://www.asl.gs/) contém 26 letras. Duas dessas letras (j e z) requerem movimento, portanto não estão incluídas no conjunto de dados de treinamento.

<img src="https://drive.google.com/uc?id=199xJ18O5FIpDAw5quyD-lfxWSMe_4Ror" style="width: 600px;">

### Kaggle

Este conjunto de dados está disponível no site [Kaggle](http://www.kaggle.com), que é um lugar fantástico para encontrar conjuntos de dados e outros recursos de aprendizagem profunda. Além de fornecer recursos como conjuntos de dados e “kernels” que são como esses notebooks, o Kaggle organiza competições das quais você pode participar, competindo com outros no treinamento de modelos de alta precisão.

Se você deseja praticar ou ver exemplos de muitos projetos de aprendizado profundo, Kaggle é um ótimo site para visitar.

## Carregando os dados

This dataset is not available via Keras in the same way that MNIST is, so let's learn how to load custom data. By the end of this section we will have `x_train`, `y_train`, `x_valid`, and `y_valid` variables as before.

### Lendo os dados

O conjunto de dados de linguagem de sinais está no formato [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) (valores separados por vírgula), a mesma estrutura de dados do Microsoft Excel e do Planilhas Google. É uma grade de linhas e colunas com rótulos na parte superior, como visto nos conjuntos de dados [train](data/asl_data/sign_mnist_train.csv) e [valid](data/asl_data/sign_mnist_valid.csv) (eles podem demorar um pouco para carregar).

Para carregar e trabalhar com os dados, usaremos uma biblioteca chamada [Pandas](https://pandas.pydata.org/), que é uma ferramenta de alto desempenho para carregar e manipular dados. Leremos os arquivos CSV em um formato chamado [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
import pandas as pd

O Pandas tem um método [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) que espera um arquivo csv e retorna um DataFrame:

In [ ]:
train_df = pd.read_csv("sign_mnist_train.csv")
valid_df = pd.read_csv("sign_mnist_valid.csv")

### Explorando a base de dados

Vamos dar uma olhada em nossos dados. Podemos usar o método [head](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) para imprimir as primeiras linhas do DataFrame. Cada linha é uma imagem que possui uma coluna `label` e também 784 valores que representam cada valor de pixel na imagem, assim como no conjunto de dados MNIST. Observe que os rótulos atualmente são valores numéricos, não letras do alfabeto:

In [ ]:
valid_df.head()

### Extraindo os rótulos

Assim como no MNIST, gostaríamos de armazenar nossos rótulos de treinamento e validação nas variáveis `y_train` e `y_valid`. Aqui criamos essas variáveis e depois excluímos os rótulos dos nossos dataframes originais, onde eles não são mais necessários:

In [ ]:
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

### Extraindo as imagens

Assim como no MNIST, gostaríamos de armazenar nossas imagens de treinamento e validação nas variáveis `x_train` e `x_valid`. Aqui criamos essas variáveis:

In [ ]:
x_train = train_df.values
x_valid = valid_df.values

### Summarizing the Training and Validation Data

Temos agora 27.455 imagens com 784 pixels cada para treinamento...

In [ ]:
x_train.shape #quantidade de linhas, quant colunas

...bem como seus rótulos correspondentes:

In [ ]:
y_train.shape #do outro lado tá vazio pq ele só pegou as linhas, e coluna nenhuma

Para validação, temos 7.172 imagens...

In [ ]:
x_valid.shape

...e seus rótulos correspondentes:

In [ ]:
y_valid.shape

## visualize os dados

Para visualizar as imagens, utilizaremos novamente a biblioteca matplotlib. Não precisamos nos preocupar com os detalhes desta visualização, mas se estiver interessado, você poderá aprender mais sobre [matplotlib](https://matplotlib.org/) posteriormente.

Observe que teremos que remodelar os dados de seu formato 1D atual de 784 pixels para um formato 2D de 28x28 pixels para dar sentido à imagem:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,40))

num_images = 20
for i in range(num_images):
    row = x_train[i]
    label = y_train[i]

    image = row.reshape(28,28)
    plt.subplot(1, num_images, i+1)
    plt.title(label, fontdict={'fontsize': 30})
    plt.axis('off')
    plt.imshow(image, cmap='gray')

## Exercício: normalize os dados de imagem

Assim como fizemos com o conjunto de dados MNIST, vamos normalizar os dados da imagem, ou seja, seus valores de pixel, em vez de ficarem entre 0 e 255 como estão atualmente:

In [ ]:
x_train.min()

In [ ]:
x_train.max()

...devem ter valores de ponto flutuante entre 0 e 1. Use a célula a seguir para trabalhar. Se você tiver dúvidas, veja a solução abaixo.

In [ ]:
x_train = x_train / 255
x_valid = x_valid / 255

## Exercício: categorize os rótulos

Assim como fizemos com o conjunto de dados MNIST, codificaremos categoricamente os rótulos. Lembre-se de que podemos usar o método [keras.utils.to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) para fazer isso, passando-lhe os valores a serem codificados e, o número de categorias para codificá-lo. Faça seu trabalho na célula abaixo. Importamos `keras` e definimos o número de categorias (24) para você.

In [ ]:
import keras
num_classes = 24

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

## Exercício: Construa o modelo

Os dados estão todos preparados, temos imagens normalizadas para treinamento e validação, bem como rótulos codificados categoricamente para treinamento e validação.

Para este exercício vamos construir um modelo sequencial. Assim como da última vez, construa um modelo que:
* Possui uma camada de entrada densa. Esta camada deve conter 512 neurônios, usar a função de ativação `relu` e esperar imagens de entrada com formato `(784,)`
* Possui uma segunda camada densa com 512 neurônios que utiliza a função de ativação `relu`
* Possui uma camada de saída densa com neurônios igual ao número de classes, usando a função de ativação `softmax`

Faça seu trabalho na célula abaixo, criando uma variável `model` para armazenar o modelo. Importamos a classe de modelo Keras [Sequental](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) e [Dense](https://www.tensorflow.org/api_docs/python /tf/keras/layers/Dense) para você começar. Revele a solução abaixo para obter uma dica:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=512, activation='relu', input_shape=(784,))) #entrada
model.add(Dense(units=512, activation='relu')) #intermediaria
model.add(Dense(units=24, activation='softmax')) #saida

## Resumo do Modelo

Execute a célula abaixo para resumir o modelo que você acabou de criar:

In [ ]:
model.summary()

## Compilando o Modelo

Vamos [compilar](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile) nosso modelo com as mesmas opções de antes, usando [entropia cruzada categórica](https://www. tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) para refletir o fato de que queremos nos enquadrar em uma das muitas categorias e medir a precisão do nosso modelo:

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Exercício: Treine o modelo

Use o método `fit` do modelo para treiná-lo por 20 épocas usando as imagens e rótulos de treinamento e validação criados acima:

In [ ]:
history = model.fit(
    x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid)
)

## Discussão: O que aconteceu?

Podemos ver que a precisão do treinamento atingiu um nível bastante alto, mas a precisão da validação não foi tão alta. O que aconteceu aqui?

Pense um pouco antes de clicar no '...' abaixo para revelar a resposta.

`#SOLUÇÃO`
Este é um exemplo de modelo aprendendo a categorizar os dados de treinamento, mas com desempenho insatisfatório em relação a novos dados nos quais não foi treinado. Essencialmente, trata-se de memorizar o conjunto de dados, mas não de obter uma compreensão robusta e geral do problema. Este é um problema comum chamado *overfitting*. Discutiremos o overfitting nas próximas duas palestras, bem como algumas maneiras de abordá-lo.

## Summary

Nesta seção, você construiu sua própria rede neural para realizar uma classificação de imagens bastante precisa. Parabéns!

Neste ponto, devemos estar familiarizados com o processo de carregamento de dados (incluindo rótulos), prepará-los, criar um modelo e, em seguida, treinar o modelo com os dados preparados.

### Limpe a memória
Antes de prosseguir, execute a seguinte célula para limpar a memória da GPU. Isso é necessário para passar para o próximo caderno.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Próximo

Agora que você construiu alguns modelos básicos e um tanto eficazes, começaremos a aprender sobre modelos mais sofisticados, incluindo *Redes Neurais Convolucionais*.